In [134]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.util import flatten
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [154]:
#read data
df = pd.read_csv("20210103_hundenamen.csv")

In [25]:
df

# since some names like "Bo" Bendy of Treegarden, has extra information other than the name,
# 2 cases are built, one with no preprocessing and one with preprocessing 

,HUNDENAME,GEBURTSJAHR_HUND,GESCHLECHT_HUND
0,Ituma,2011,w
1,"""Bo"" Bendy of Treegarden",2020,m
2,"""Bobby"" Lord Sinclair",2009,m
3,"""Buddy"" Fortheringhay's J.",2011,m
4,"""Fly"" Showring i fly for you",2015,w
...,...,...,...
8569,unbekannt,2010,w
8570,unbekannt,2011,m
8571,unbekannt,2018,m
8572,unbekannt,2018,m


In [77]:
# install python library

pip install python-Levenshtein

     |████████████████████████████████| 51kB 2.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149808 sha256=d8747a0c5032902d7a24c8fce73146a6550c635509f0888e99182ae9f1c46404
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
# import 
from Levenshtein import distance as levenshtein_distance

In [123]:
# CASE 1
# no preprocessing

Levenshtein_distance = []

for i in df.HUNDENAME.values:
  Levenshtein_distance.append(levenshtein_distance('luca',i))

df['Levenshtein_distance'] = Levenshtein_distance
df[df.Levenshtein_distance==1].HUNDENAME.unique()

# ['Cuca', 'Luca', 'Yuca'] are all names that have a Levenshtein distance of 1 to "Luca"
# we were able to find very less names with out preprocessing

array(['Cuca', 'Luca', 'Yuca'], dtype=object)

In [160]:
# CASE 2
# splitting bigger names into small names and de-capitalizing all names
# in simple words, cleaning messy text

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

#Include: personal pronouns and possessive pronouns gender indicators
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text


df['cleaned_Data'] = df['HUNDENAME'].apply(lambda x: clean_text(x))
temp = df.cleaned_Data.map(lambda x: x.split())

# creating a new list of dog names from the dataset
dog_names = []
for i in temp.values:
  dog_names.append(i)

dog_names_edited = list(set(flatten(dog_names))) # removes repetitions of names and outputs flattened list

df_edit = pd.DataFrame(dog_names_edited,columns=["dognames"]) # create new dataframe with names

Levenshtein_distance = []

for i in df_edit.dognames.values:
  Levenshtein_distance.append(levenshtein_distance('luca',i))

df_edit['Levenshtein_distance'] = Levenshtein_distance
df_edit[df_edit.Levenshtein_distance==1].dognames.unique()

# ['cuca', 'lula', 'luba', 'luna', 'lucy', 'yuca', 'lucas', 'luce',
#        'luka', 'luck', 'lua', 'lucia', 'lupa', 'luma']

# total 14 names are found which is better than case 1

# NOTE: all names are in lower cases 

array(['cuca', 'lula', 'luba', 'luna', 'lucy', 'yuca', 'lucas', 'luce',
       'luka', 'luck', 'lua', 'lucia', 'lupa', 'luma'], dtype=object)